In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

# Choose an LLM Model

In [12]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

# Create your tools

In [17]:
from langchain_core.tools import tool


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


In [30]:
tools_list = [multiply]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

# Bind tools with LLMs

In [31]:
# we create a tool calling Agent by binding a list of tools to the llm

llm_with_tools = llm.bind_tools(tools_list)

In [32]:

response = llm_with_tools.invoke("What is 10 times 40?")

response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rmqd87pfq', 'function': {'arguments': '{"a":10,"b":40}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 220, 'total_tokens': 236, 'completion_time': 0.047777684, 'prompt_time': 0.024325807, 'queue_time': 0.045898713, 'total_time': 0.072103491}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0379601e-347e-4fc9-b38a-893aa4e03125-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 40}, 'id': 'rmqd87pfq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 220, 'output_tokens': 16, 'total_tokens': 236})

In [20]:
# Exercise: Modify the prompt in various ways

# Tool Invokation

Tool calling is NOT tool execution

In [33]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 10, 'b': 40},
  'id': 'rmqd87pfq',
  'type': 'tool_call'}]

In [40]:
tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0])

ToolMessage(content='400.0', name='multiply', tool_call_id='rmqd87pfq')